In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Pre-processing

In [62]:
df1 = pd.read_csv("../outputs/2022_national_health.csv")
df2 = pd.read_csv("../outputs/2023_national_health.csv")
df3 = pd.read_csv("../outputs/2024_national_health.csv")

In [63]:
df1['year'] = 2022
df2['year'] = 2023
df3['year'] = 2024

In [64]:
df = pd.concat([df1, df2, df3], ignore_index = True)

In [65]:
df = df[~df['COUNTY_NAME'].isna()]

In [66]:
df = df[df['Diabetes'] != -1]

In [67]:
len(county)

9434

In [68]:
mean_obesity = np.mean([x for x in df['Obesity'].tolist() if x != -1])
df['Obesity'] = df['Obesity'].apply(lambda x : x if x != -1 else mean_obesity)

mean_smoking = np.mean([x for x in df['Adult Smoking'].tolist() if x != -1])
df['Adult Smoking'] = df['Adult Smoking'].apply(lambda x : x if x != -1 else mean_smoking)

mean_inactivity = np.mean([x for x in df['Physical Inactivity'].tolist() if x != -1])
df['Physical Inactivity'] = df['Physical Inactivity'].apply(lambda x : x if x != -1 else mean_inactivity)

mean_drinking = np.mean([x for x in df['Excessive Drinking'].tolist() if x != -1])
df['Excessive Drinking'] = df['Excessive Drinking'].apply(lambda x : x if x != -1 else mean_drinking)

mean_mental = np.mean([x for x in df['Poor Mental Health'].tolist() if x != -1])
df['Poor Mental Health'] = df['Poor Mental Health'].apply(lambda x : x if x != -1 else mean_mental)

# Training

In [70]:
features = ["Obesity", "Adult Smoking", "Physical Inactivity", "Excessive Drinking", "Poor Mental Health"]
target = "Diabetes"

In [71]:
X = df[features]
y = df[target]

In [79]:
X.head(1)

,Obesity,Adult Smoking,Physical Inactivity,Excessive Drinking,Poor Mental Health
1,35.0,20.0,32.0,16.0,17.0


In [80]:
y.head(1)

1    11.0
Name: Diabetes, dtype: float64

In [72]:
regressor = LinearRegression()
regressor.fit(X, y)

LinearRegression()

In [73]:
regressor.score(X, y)

0.7219056647318691

In [74]:
regressor.coef_

array([ 0.1109634 , -0.04214315,  0.22483687, -0.13264213,  0.07376137])

In [75]:
regressor.intercept_

np.float64(2.4683950600895077)

# Fitted Value

In [125]:
df3.head(2)

,STATE_NAME,COUNTY_NAME,FIPS,Obesity,Adult Smoking,Physical Inactivity,Excessive Drinking,Poor Mental Health,Diabetes,year
0,Alabama,NaN,1000,41.0,18.0,30.0,14.0,20.0,13.0,2024
1,Alabama,Autauga,1001,39.0,17.0,29.0,15.0,18.0,11.0,2024


In [126]:
state = df3[df3['COUNTY_NAME'].isna()]

In [127]:
len(state)

51

In [128]:
state_X = state[features]

In [129]:
state_X.head(1)

,Obesity,Adult Smoking,Physical Inactivity,Excessive Drinking,Poor Mental Health
0,41.0,18.0,30.0,14.0,20.0


In [130]:
len(state_X)

51

In [131]:
y_hat = regressor.predict(state_X)

In [132]:
state['Diabetes_hat'] = y_hat

/var/folders/j2/nhdcqck90txbdvmjx442mm_00000gn/T/ipykernel_35643/814113338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state['Diabetes_hat'] = y_hat


# Warning

In [133]:
state.head()

,STATE_NAME,COUNTY_NAME,FIPS,Obesity,Adult Smoking,Physical Inactivity,Excessive Drinking,Poor Mental Health,Diabetes,year,Diabetes_hat
0,Alabama,NaN,1000,41.0,18.0,30.0,14.0,20.0,13.0,2024,12.622662
68,Alaska,NaN,2000,33.0,17.0,20.0,20.0,14.0,8.0,2024,8.290308
104,Arizona,NaN,4000,32.0,14.0,22.0,19.0,16.0,10.0,2024,9.035613
120,Arkansas,NaN,5000,39.0,22.0,30.0,15.0,19.0,11.0,2024,12.025759
196,California,NaN,6000,28.0,9.0,20.0,17.0,15.0,11.0,2024,8.544324


In [134]:
state['diff'] = state['Diabetes_hat'] - state['Diabetes']

/var/folders/j2/nhdcqck90txbdvmjx442mm_00000gn/T/ipykernel_35643/1888098546.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state['diff'] = state['Diabetes_hat'] - state['Diabetes']


In [135]:
max(state['diff'])

1.5248129860278858

In [136]:
diff_sd = np.std(state['diff'])
diff_sd

np.float64(0.9049014727676141)

In [137]:
diff_mean = np.mean(state['diff'])
diff_mean

np.float64(-0.32928104473188324)

In [138]:
state['warning'] = state['diff'].apply(lambda x : 1 if x > 0 else 0)

/var/folders/j2/nhdcqck90txbdvmjx442mm_00000gn/T/ipykernel_35643/3781873470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state['warning'] = state['diff'].apply(lambda x : 1 if x > 0 else 0)


In [139]:
warning_state = state[state['warning'] == 1]

In [140]:
len(warning_state)

22

In [141]:
state['warning2'] = state['diff'].apply(lambda x : 1 if x > diff_mean + diff_sd else 0)

/var/folders/j2/nhdcqck90txbdvmjx442mm_00000gn/T/ipykernel_35643/2037529455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state['warning2'] = state['diff'].apply(lambda x : 1 if x > diff_mean + diff_sd else 0)


In [142]:
warning_state_2 = state[state['warning2'] == 1]

In [143]:
warning_state_2

,STATE_NAME,COUNTY_NAME,FIPS,Obesity,Adult Smoking,Physical Inactivity,Excessive Drinking,Poor Mental Health,Diabetes,year,Diabetes_hat,diff,warning,warning2
120,Arkansas,NaN,5000,39.0,22.0,30.0,15.0,19.0,11.0,2024,12.025759,1.025759,1,1
1202,Maine,NaN,23000,32.0,17.0,25.0,20.0,17.0,8.0,2024,9.524813,1.524813,1,1
1799,New Hampshire,NaN,33000,31.0,13.0,19.0,19.0,16.0,7.0,2024,8.292282,1.292282,1,1
2828,Utah,NaN,49000,32.0,7.0,18.0,12.0,16.0,8.0,2024,9.359762,1.359762,1,1
3177,Wyoming,NaN,56000,32.0,17.0,22.0,19.0,16.0,8.0,2024,8.909183,0.909183,1,1


In [144]:
state['warning_final'] = state['warning'] + state['warning2']

/var/folders/j2/nhdcqck90txbdvmjx442mm_00000gn/T/ipykernel_35643/1682439484.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state['warning_final'] = state['warning'] + state['warning2']


# Output

In [145]:
state = state[['FIPS', 'STATE_NAME', 'Diabetes', 'Diabetes_hat', 'warning_final']]

In [146]:
state = state.rename(columns = {"warning_final" : "Warning", 
                                "Diabetes" : "Diabetes_Actual",
                                "Diabetes_hat" : "Diabetes_Predicted"
                               }
                    )

In [148]:
state['FIPS'] = state['FIPS'].apply(lambda x : str(int(x / 1000)))

In [150]:
state['FIPS'] = state['FIPS'].apply(lambda x : "0" + x if len(x) == 1 else x)

In [151]:
state

,FIPS,STATE_NAME,Diabetes_Actual,Diabetes_Predicted,Warning
0,01,Alabama,13.0,12.622662,0
68,02,Alaska,8.0,8.290308,1
104,04,Arizona,10.0,9.035613,0
120,05,Arkansas,11.0,12.025759,2
196,06,California,11.0,8.544324,0
255,08,Colorado,7.0,6.879925,0
320,09,Connecticut,9.0,9.052936,1
329,10,Delaware,10.0,10.407291,1
333,11,District of Columbia,8.0,6.225544,0
335,12,Florida,10.0,9.450820,0


In [152]:
state_warning = state.to_json("../outputs/state_warning.json", orient="records")